In [28]:
import yfinance as yf
from proxy import enable
from lorentzian import lorentzian_distance, lorentzian_k_nearest_neighbor, Diction, KNode, test_data_by_df, \
    prediction2node
from features import n_rsi, n_adx, n_cci, n_wt, HLC3, FeaturesX, FeaturesDefinition
import ta.wrapper

enable()

In [29]:
ticker = yf.Ticker("AAPL")

In [30]:
his = ticker.history(period="max", interval="1d")

In [31]:
his['SMA20'] = ta.wrapper.SMAIndicator(his['Close'], window=20).sma_indicator()
his['EMA20'] = ta.wrapper.EMAIndicator(his['Close'], window=20).ema_indicator()

In [32]:
his['Close-SMA'] = his["Close"] - his["SMA20"]
his['Close-EMA'] = his["Close"] - his["EMA20"]

In [33]:
features_definition = FeaturesDefinition.default(high=his['High'], close=his['Close'], open=his['Open'], low=his['Low'], vol=his['Volume'])
his = features_definition.calculate(his)

In [34]:
test_data = test_data_by_df(his, features_definition, future_count=4)

In [35]:
start_index = int(len(test_data) * 0.9)
train = test_data[:start_index]
test = test_data[start_index:]

In [36]:
def buy_signal(the_current, the_test):
    return (
            # the_test.info['Close-SMA'] > 0 and 
            the_test.info['Close-EMA'] > 0 
            # and the_test.info['Close'] >the_test.info['Open'] 
            and the_current.diction == Diction.LONG and 8 >= the_current.prediction > 2)


def sell_signal(the_current, the_test):
    return (
            # the_test.info['Close-SMA'] < 0 and 
            the_test.info['Close-EMA'] < 0 
            # and the_test.info['Close'] < the_test.info['Open'] 
            and the_current.diction == Diction.SHORT and -8 <= the_current.prediction < -2)

In [37]:
win_num = 0
buy_win_num = 0
sell_win_num = 0
buy_num = 0
sell_num = 0
pos_index = 1
total = 0
for k_node in test:
    current_node = prediction2node(k_node.info, train, features_definition, k=12)
    if buy_signal(current_node, k_node):
        buy_num = buy_num + 1
        if current_node.diction == k_node.diction:
            win_num = win_num + 1
            buy_win_num = buy_win_num + 1
    elif sell_signal(current_node, k_node):
        sell_num = sell_num + 1
        if current_node.diction == k_node.diction:
            win_num = win_num + 1
            sell_win_num = sell_win_num + 1

    total = buy_num + sell_num

    win_rate = win_num / total * 100 if total != 0 else 0
    buy_win_rate = buy_win_num / buy_num * 100 if buy_num != 0 else 0
    sell_win_rate = sell_win_num / sell_num * 100 if sell_num != 0 else 0
    print(
        f'progress: {pos_index}/{len(test)} | buy: {buy_num} | sell: {sell_num} | Total: {total} | Win: {win_num} | Win Rate: {win_rate:.2f}% | Buy Win Rate: {buy_win_rate:.2f}% | Sell Win Rate: {sell_win_rate:.2f}%',
        end='\r', flush=True)
    pos_index = pos_index + 1

print('\n\ntest end.\n')
print(
    f'Win Rate: {win_num / total * 100:.2f}% | Win: {win_num} | Total: {total} | buy: {buy_num} | sell: {sell_num}')

8/1081 | buy: 2 | sell: 0 | Total: 2 | Win: 1 | Win Rate: 50.00% | Buy Win Rate: 50.00% | Sell Win Rate: 0.00%progress: 10/1081 | buy: 3 | sell: 0 | Total: 3 | Win: 1 | Win Rate: 33.33% | Buy Win Rate: 33.33% | Sell Win Rate: 0.00%progress: 13/1081 | buy: 3 | sell: 0 | Total: 3 | Win: 1 | Win Rate: 33.33% | Buy Win Rate: 33.33% | Sell Win Rate: 0.00%progress: 15/1081 | buy: 3 | sell: 0 | Total: 3 | Win: 1 | Win Rate: 33.33% | Buy Win Rate: 33.33% | Sell Win Rate: 0.00%progress: 17/1081 | buy: 3 | sell: 0 | Total: 3 | Win: 1 | Win Rate: 33.33% | Buy Win Rate: 33.33% | Sell Win Rate: 0.00%progress: 19/1081 | buy: 3 | sell: 0 | Total: 3 | Win: 1 | Win Rate: 33.33% | Buy Win Rate: 33.33% | Sell Win Rate: 0.00%progress: 21/1081 | buy: 3 | sell: 0 | Total: 3 | Win: 1 | Win Rate: 33.33% | Buy Win Rate: 33.33% | Sell Win Rate: 0.00%progress: 23/1081 | buy: 4 | sell: 0 | Total: 4 | Win: 2 | Win Rate: 50.00% | Buy Win Rate: 50.00% | Sell Win Rate: 0.00%progress: 25/1081 | buy: 4 | sell: 0 | Tota

In [16]:
node = prediction2node(his.iloc[-1], test_data, features_definition)

In [ ]:
[neighbor.node.prediction for neighbor in node.neighbor_nodes]